In [1]:
#데이터 불러오기 
import pandas as pd
boodongsan_data = pd.read_csv('../final_data/new_boodongsan_data_utf.csv', encoding='utf-8')
boodongsan_data.head()

#데이터 전처리 

#소재지 없는거 제거 
dataframe = pd.DataFrame()

#중복값 제거 
boodongsan_data = boodongsan_data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
print(len(boodongsan_data))
#소재지 없는곳 제거
boodongsan_data = boodongsan_data[~boodongsan_data['소재지'].isnull()]
print(len(boodongsan_data))


6885
5008


In [2]:
boodongsan_data


,x,y,마트,학교,사진,제목,가격,소제목,소재지,매물특징,...,총세대수,용도지역,주구조,전세금 반환보증,재건축/재개발,준공인가일,사용검사일,대지/연면적,건축/전용면적,지상층/지하층
0,126.963506,37.477090,0,0,0,일반원룸6층,"전세2억 2,000(2,582만원/3.3㎡)",해당층/총층: 6/10층오픈형/단층 전용 면적:17.38㎡,서울시 관악구 봉천동 1660-12,"신축, 창이 넓고, 깔끔하고, 혼자 살기 좋은 방.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126.963506,37.477090,0,0,0,오피스텔 o리더스빌o6층,"월세1,000/63",해당층/총층: 6/6층분리형/단층 전용 면적:20㎡,서울시 관악구 봉천동 1660-2,낙성대역 초역세권 대로변 오피스텔 1년계약,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,126.963506,37.477090,0,0,0,일반원룸3층,"월세1,000/70",해당층/총층: 3/6층분리형/단층 전용 면적:21.48㎡,서울시 관악구 봉천동 1660-2,낙성대역 초역세권 오피스텔 월세,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,126.963506,37.477090,0,0,0,빌라B1층,"전세1억 3,000(1,294만원/3.3㎡)",방 2개사용승인일:1993.08.05주차불가 공급/전용 면적:33.22㎡/25.17㎡,서울시 관악구 봉천동 1686-6,낙성대 초역세 수리된 저렴한 빌라 2룸 전세 13000만원,...,6세대,-,-,"예상 보증료382,570원네이버페이 최대1만원 적립상품 안내",NaN,NaN,NaN,NaN,NaN,NaN
7,126.963506,37.477090,0,0,0,빌라B1층,"전세1억 3,500(1,288만원/3.3㎡)",방 2개사용승인일:1993.08.05주차불가 공급/전용 면적:34.66㎡/26.61㎡,서울시 관악구 봉천동 1686-6,옵션OK 애완가능 위치좋은 빌라 투룸,...,6세대,-,-,"예상 보증료397,290원네이버페이 최대1만원 적립상품 안내",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7783,126.941670,37.484370,0,1,0,다가구1층,"월세1,000/65",해당층/총층: 1/4층방 2개남서향 공급/전용 면적:33.05㎡/26.44㎡,서울시 관악구 봉천동 934-2,"투룸식 깔끔하고, 살기 좋은 방.",...,-,-,-,NaN,-,NaN,NaN,NaN,NaN,NaN
7784,126.941670,37.484370,0,1,0,빌라2층,"월세5,000/75",방 2개사용승인일:2002.09.10주차가능 공급/전용 면적:58.74㎡/45.15㎡,서울시 관악구 봉천동 932-22,"투룸, 거실, 베란다 주차가능, 즉시입주 가능",...,-,-,-,"예상 보증료147,140원네이버페이 최대1만원 적립상품 안내",NaN,NaN,NaN,NaN,NaN,NaN
7789,126.941670,37.484370,0,1,0,빌라2층,"월세5,000/75",방 2개사용승인일:2002.09.10주차가능 공급/전용 면적:51.57㎡/45.12㎡,서울시 관악구 봉천동 932-22,초역세권 내부 수리된 큰 2룸과 거실 넓은 주거공간 희망하시는분,...,-,-,-,"예상 보증료147,140원네이버페이 최대1만원 적립상품 안내",NaN,NaN,NaN,NaN,NaN,NaN
7790,126.941670,37.484370,0,1,0,다가구 다솜하우스3층,"전세2억 3,000(1,917만원/3.3㎡)",해당층/총층: 3/4층방 2개남동향 공급/전용 면적:39.66㎡/36.36㎡,서울시 관악구 봉천동 908-63,2호선 봉천역 초역세권 넓은 풀옵션 투룸6,...,-,-,-,NaN,-,NaN,NaN,NaN,NaN,NaN


In [3]:
#데이터 불러오기 
import pandas as pd
boodongsan_data = pd.read_csv('../final_data/new_boodongsan_data_utf.csv', encoding='utf-8')
#소재지 없는거 제거 
dataframe = pd.DataFrame()

#중복값 제거 
boodongsan_data = boodongsan_data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
print(len(boodongsan_data))
#소재지 없는곳 제거
boodongsan_data = boodongsan_data[~boodongsan_data['소재지'].isnull()]
print(len(boodongsan_data))

6885
5008


In [4]:
#보증금, 월세, 관리비 합치기 
import re
import numpy as np
def deposit(x):
    #월세에 억이 박히는 경우도 이있다 
    if '월세' in x:
        wol = x.split('/')[0]
        wol = re.sub(r"[^0-9]", "", wol)
        return int(wol)
    elif '전세' in x:
        jun = x.split('(')[0]
        uk = 0
        if '억' in jun:
            uk = jun.split('억')[0]
            jun = jun.split('억')[1]
            uk = re.sub(r"[^0-9]", "", uk)
            uk = int(uk)
        jun = re.sub(r"[^0-9]", "", jun)
        if not jun:
            jun = 0
        return uk*10000 + int(jun)

def monthly(x):
    if '월세' in x:
        if '(' in x:
            x = x.split('(')[0]

        x = x.split('/')[1][:4]
        x = re.sub(r"[^0-9]", "", x)
    else:
        x = 0
    
    return int(x)
def cost(x):
    x = str(x)
    x = re.sub(r"[^0-9]", "", x)
    #관리비 이상치 처리 1이하인값  처리 
    try:
        if int(x) < 1 :
            
            x = x[-2:]
    except:
        return 0
    return int(x)

#월세 전세 여부
def charter_montly(x):
    if '월세' in x:
        k = '월세'
    elif '전세' in x:
        k = '전세'
        
    #단기임대...
    else:
        k = '?'
    return k

boodongsan_data['보증금'] = boodongsan_data['가격'].apply(lambda x : deposit(x))
boodongsan_data['월전세'] = boodongsan_data['가격'].apply(lambda x : charter_montly(x))
#보증금 1년 이자 5%
boodongsan_data['보증금_5'] = boodongsan_data['보증금'] * (0.05/12)
boodongsan_data['월세'] = boodongsan_data['가격'].apply(lambda x : monthly(x))
boodongsan_data['월관리비'].fillna(0)
boodongsan_data['월관리비'] = boodongsan_data['월관리비'].apply(lambda x : cost(x))
#총 월비용 
boodongsan_data['월비용'] = boodongsan_data['보증금_5'] + boodongsan_data['월세'] + boodongsan_data['월관리비']

In [5]:
#전용면적 구하기  상가건물은 계약/면적으로 되어있다 
boodongsan_data['전용면적']  = boodongsan_data['계약/전용면적'].fillna('') + boodongsan_data['공급/전용면적'].fillna('')
boodongsan_data = boodongsan_data.dropna(subset = ['전용면적'], axis = 0)
def area(raw):
    print(raw)
    try:
        x = raw.split('/')[1].split('(')[0].rstrip('㎡')
    except:
        return(np.nan)
    try:
        return float(x)
    except:
        return(np.nan)
    

#전용면적 추출
boodongsan_data['전용면적']  = boodongsan_data['전용면적'].apply(area)
boodongsan_data = boodongsan_data.dropna(subset = ['전용면적'], axis = 0)


28.18㎡/17.38㎡(전용률62%)
22㎡/20㎡(전용률91%)
-㎡/21.48㎡(전용률-%)
33.22㎡/25.17㎡(전용률76%)
34.66㎡/26.61㎡(전용률77%)
33.22㎡/25.17㎡(전용률76%)
34.66㎡/26.61㎡(전용률77%)
24.43㎡/16.2㎡(전용률66%)
24.43㎡/16.2㎡(전용률66%)
26.4㎡/19.8㎡(전용률75%)
23.1㎡/23.1㎡(전용률100%)
23.1㎡/19.8㎡(전용률86%)
23.1㎡/19.8㎡(전용률86%)
19.8㎡/16.5㎡(전용률83%)
19.83㎡/17㎡(전용률86%)
36.36㎡/33.05㎡(전용률91%)
-㎡/23.14㎡(전용률-%)
24.7㎡/24.7㎡(전용률100%)
23.2㎡/23.2㎡(전용률100%)
19.8㎡/19.8㎡(전용률100%)
23.1㎡/19.8㎡(전용률86%)
26.4㎡/19.8㎡(전용률75%)
21.49㎡/19.83㎡(전용률92%)
22.87㎡/15㎡(전용률66%)
23.14㎡/19.83㎡(전용률86%)
-㎡/15㎡(전용률-%)
18.1㎡/16.5㎡(전용률91%)
18.1㎡/16.5㎡(전용률91%)
18㎡/16㎡(전용률89%)
43.68㎡/24.92㎡(전용률57%)
42.33㎡/22.17㎡(전용률52%)
29.7㎡/24.7㎡(전용률83%)
29.7㎡/24.7㎡(전용률83%)
23.1㎡/20㎡(전용률87%)
21.4㎡/21.4㎡(전용률100%)
23.1㎡/19.8㎡(전용률86%)
16㎡/16㎡(전용률100%)
27.2㎡/16.83㎡(전용률62%)
24.22㎡/18.09㎡(전용률75%)
20.98㎡/13.83㎡(전용률66%)
22.53㎡/16.83㎡(전용률75%)
22.53㎡/16.83㎡(전용률75%)
75.68㎡/39.67㎡(전용률52%)
49.58㎡/36.36㎡(전용률73%)
75.68㎡/39.67㎡(전용률52%)
33.06㎡/26.44㎡(전용률80%)
49.58㎡/36.22㎡(전용률73%)
75.68㎡/37㎡(전용률49%)
20㎡/18㎡(전용률90%)
25㎡/20

In [6]:
# 좌표 만드는 코드 // 주소만드는 코드  동까지만 표시
def dong_data(x):
    x = ' '.join(x.split()[0:3])
    return x

boodongsan_data['시구동'] =  boodongsan_data['소재지'].apply(dong_data)

In [7]:
#단기임대 삭제
boodongsan_data = boodongsan_data.loc[~boodongsan_data['월전세'].isin(['?'])]

In [8]:
import plotly.express as px
# x 전용 면적, y 월비용, color 위치, symbol 월세, 전세, size 보증금
# 점시각화
fig = px.scatter(boodongsan_data, x = '월비용',
                 y = '전용면적',
                 color = '시구동', 
                 color_discrete_sequence=px.colors.qualitative.Light24,
                 size = '보증금',
                 hover_name='매물번호',
                 hover_data=['소재지'],
                 symbol= '월전세')

# fig.update_traces(hovertemplate = '%{시구동}')
fig.show()

#저장
import plotly.io as po
po.write_html(fig, file='test.html', auto_open=True)

In [11]:
# 월비용 100이하만 추출
boodongsan_data_100  = boodongsan_data.loc[boodongsan_data['월비용'] <= 100]
boodongsan_data_100

,x,y,마트,학교,사진,제목,가격,소제목,소재지,매물특징,...,대지/연면적,건축/전용면적,지상층/지하층,보증금,월전세,보증금_5,월세,월비용,전용면적,시구동
4,126.963506,37.477090,0,0,0,오피스텔 o리더스빌o6층,"월세1,000/63",해당층/총층: 6/6층분리형/단층 전용 면적:20㎡,서울시 관악구 봉천동 1660-2,낙성대역 초역세권 대로변 오피스텔 1년계약,...,NaN,NaN,NaN,1000.0,월세,4.166667,63,77.166667,20.00,서울시 관악구 봉천동
5,126.963506,37.477090,0,0,0,일반원룸3층,"월세1,000/70",해당층/총층: 3/6층분리형/단층 전용 면적:21.48㎡,서울시 관악구 봉천동 1660-2,낙성대역 초역세권 오피스텔 월세,...,NaN,NaN,NaN,1000.0,월세,4.166667,70,82.166667,21.48,서울시 관악구 봉천동
6,126.963506,37.477090,0,0,0,빌라B1층,"전세1억 3,000(1,294만원/3.3㎡)",방 2개사용승인일:1993.08.05주차불가 공급/전용 면적:33.22㎡/25.17㎡,서울시 관악구 봉천동 1686-6,낙성대 초역세 수리된 저렴한 빌라 2룸 전세 13000만원,...,NaN,NaN,NaN,13000.0,전세,54.166667,0,54.166667,25.17,서울시 관악구 봉천동
7,126.963506,37.477090,0,0,0,빌라B1층,"전세1억 3,500(1,288만원/3.3㎡)",방 2개사용승인일:1993.08.05주차불가 공급/전용 면적:34.66㎡/26.61㎡,서울시 관악구 봉천동 1686-6,옵션OK 애완가능 위치좋은 빌라 투룸,...,NaN,NaN,NaN,13500.0,전세,56.250000,0,56.250000,26.61,서울시 관악구 봉천동
8,126.963506,37.477090,0,0,0,빌라B1층,"전세1억 3,000(1,294만원/3.3㎡)",방 2개사용승인일:1993.08.05주차불가 공급/전용 면적:33.22㎡/25.17㎡,서울시 관악구 봉천동 1686-6,역1분. 전세대출.수리된 투룸,...,NaN,NaN,NaN,13000.0,전세,54.166667,0,54.166667,25.17,서울시 관악구 봉천동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,126.941670,37.484370,0,1,0,다가구저층,"월세1,000/65",해당층/총층: 저/3층방 2개남향 공급/전용 면적:55㎡/50㎡,서울시 관악구 봉천동 934-26,"봉천역역세권 2룸,가성비최고",...,NaN,NaN,NaN,1000.0,월세,4.166667,65,71.166667,50.00,서울시 관악구 봉천동
7783,126.941670,37.484370,0,1,0,다가구1층,"월세1,000/65",해당층/총층: 1/4층방 2개남서향 공급/전용 면적:33.05㎡/26.44㎡,서울시 관악구 봉천동 934-2,"투룸식 깔끔하고, 살기 좋은 방.",...,NaN,NaN,NaN,1000.0,월세,4.166667,65,72.166667,26.44,서울시 관악구 봉천동
7784,126.941670,37.484370,0,1,0,빌라2층,"월세5,000/75",방 2개사용승인일:2002.09.10주차가능 공급/전용 면적:58.74㎡/45.15㎡,서울시 관악구 봉천동 932-22,"투룸, 거실, 베란다 주차가능, 즉시입주 가능",...,NaN,NaN,NaN,5000.0,월세,20.833333,75,97.833333,45.15,서울시 관악구 봉천동
7789,126.941670,37.484370,0,1,0,빌라2층,"월세5,000/75",방 2개사용승인일:2002.09.10주차가능 공급/전용 면적:51.57㎡/45.12㎡,서울시 관악구 봉천동 932-22,초역세권 내부 수리된 큰 2룸과 거실 넓은 주거공간 희망하시는분,...,NaN,NaN,NaN,5000.0,월세,20.833333,75,97.833333,45.12,서울시 관악구 봉천동


In [12]:
import plotly.express as px
# x 전용 면적, y 월비용, color 위치, symbol 월세, 전세, size 보증금
# 점시각화
fig = px.scatter(boodongsan_data_100, x = '월비용',
                 y = '전용면적',
                 color = '시구동', 
                 color_discrete_sequence=px.colors.qualitative.Light24,
                 size = '보증금',
                 hover_name='매물번호',
                 hover_data=['소재지'],
                 symbol= '월전세')

# fig.update_traces(hovertemplate = '%{시구동}')
fig.show()

#저장
import plotly.io as po
po.write_html(fig, file='boodongsan_100.html', auto_open=True)

In [14]:
boodongsan_data_100['평당월비용'] = boodongsan_data_100['월비용'] / boodongsan_data_100['전용면적']
boodongsan_data_100['평당월비용'] 

/var/folders/4h/pt3p68k541577dsnm7h_t8dh0000gn/T/ipykernel_55673/2362604856.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4       3.858333
5       3.825264
6       2.152033
7       2.113867
8       2.152033
          ...   
7782    1.423333
7783    2.729450
7784    2.166851
7789    2.168292
7791    3.364198
Name: 평당월비용, Length: 3505, dtype: float64

In [26]:
#막대그래프_전월세 합친거 #월1m'2당비용 만들기 
#시군구당 평당 비용 확인
len(boodongsan_data_100['시구동'].unique())
#데이터 만들어주자
group_all = boodongsan_data_100[['시구동','평당월비용']].groupby(['시구동']).mean()
group_all = group_all.reset_index()
group_all = group_all.sort_values(by = ['평당월비용'])

fig = px.bar(x = group_all['시구동'], y = group_all['평당월비용']* 3)
fig.show()

In [51]:
#막대그래프_전월세 나눈거
#막대그래프_전월세 합친거 #월1m'2당비용 만들기 
#시군구당 평당 비용 확인
#데이터 만들어주자
boodongsan_data_100['시구동_월전세'] = boodongsan_data_100['시구동'] + '_' + boodongsan_data_100['월전세']

group_split = boodongsan_data_100[['시구동_월전세','평당월비용']].groupby(['시구동_월전세']).mean()
group_split = group_split.reset_index()

group_split['시구동'] = group_split['시구동_월전세'].str.split('_').str[0]
group_split['월전세'] = group_split['시구동_월전세'].str.split('_').str[1]
group_split = group_split.sort_values(by = ['평당월비용'])

fig = px.bar( x = group_split['시구동'], y= group_split['평당월비용'] * 3, color = group_split['월전세'], barmode = 'group')
fig.show()

/var/folders/4h/pt3p68k541577dsnm7h_t8dh0000gn/T/ipykernel_55673/3746061955.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [157]:
# 공인 중개사 찾아보기
# 전용면적 20 이상 월 60이하 
broker = boodongsan_data[(boodongsan_data['전용면적'] > 20) & (boodongsan_data['월비용'] < 60)]
broker_count = pd.value_counts(broker['중개사'], sort=True)
broker_count_data = pd.DataFrame()
broker_count_data['raw'] = broker_count.index
broker_count_data['count'] = broker_count.values
broker_count_data

,raw,count
0,거송부동산공인중개사사무소길찾기대표서태석등록번호11230-2019-00153소재지서울...,8
1,한빛부동산공인중개사사무소길찾기대표이경생등록번호11260-2021-00110소재지서울...,8
2,늘봄공인중개사사무소길찾기대표한은혜등록번호1162020190229소재지서울특별시 관악...,7
3,상봉역7번출구부동산 공인중개사무소톡톡문의길찾기대표이경희등록번호11260-2022-0...,4
4,타임공인중개사사무소길찾기대표박정수등록번호11260-2017-00081소재지서울특별시...,3
...,...,...
60,성공공인중개사사무소길찾기대표김수환등록번호11260-2018-00081소재지서울특별시...,1
61,대박공인중개사사무소길찾기대표최준원등록번호11260-2021-00077\t소재지서울특...,1
62,우리공인중개사사무소길찾기대표박은숙등록번호11260-2021-00120소재지서울특별시...,1
63,괜찮은공인중개사사무소톡톡문의길찾기대표원소희등록번호11260-2020-00097소재지...,1


In [ ]:
test = boodongsan_data_100[['x','y','매물번호','월비용','전용면적','월전세','보증금','소재지','평당월비용']]

In [158]:
# 전처리 
# 사무소, 위치
def joongaeso(x):
    samuso = ''
    if '길찾기' in x:
        samuso = x.split('길찾기')[0]
    elif '사무소' in x:
        samuso = x.split('사무소')[0]
    sojaeje = x.split('소재지')[1].split('전화')[0]

    return [samuso,sojaeje]

broker_count_data['사무소'],broker_count_data['위치']  = zip(*broker_count_data['raw'].apply(lambda x : joongaeso(x)))

In [163]:
import plotly.express as px
#가지고 있는거 5개 이상만 추출하여 막대 그래프를 만들어준다 
fig = px.bar(broker_count_data[broker_count_data['count'] > 3], x = '사무소', y = 'count', color='위치')
fig.show()

In [164]:
import plotly.express as px
# x 전용 면적, y 월비용, color 위치, symbol 월세, 전세, size 보증금
fig = px.scatter(boodongsan_data[boodongsan_data['중개사'].str.contains('늘봄|한빛|거송|7번출구')], x = '월비용',
                 y = '전용면적',
                 color = '시구동', 
                 color_discrete_sequence=px.colors.qualitative.Light24,
                 size = '보증금',
                 hover_name='매물번호',
                 hover_data=['소재지'],
                 symbol= '월전세')

# fig.update_traces(hovertemplate = '%{시구동}')
fig.show()

In [8]:
#데이터 시각화 완성 링크로 바로 쉽게 가는 방법 개선하면 될듯 -> 링크 + 좌표값 + 매물번호
import chart_studio

username = 'wodud2970' # your username
api_key = '' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)


In [9]:
import chart_studio.plotly as py
py.plot(fig, filename = 'boodongsan', auto_open=True)

'https://plotly.com/~wodud2970/1/'

In [10]:
import chart_studio.tools as tls
tls.get_embed('https://plotly.com/~wodud2970/1/') #change to your url

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~wodud2970/1.embed" height="525" width="100%"></iframe>'